In [60]:
#This model runs the steady-state simulation of a Plug flow reactor for the oxidation of exhaust gas
#emissions (CH4, CO, C2H6, C2H4, H2) over a Pt(111) surface. 
# Refer to the following preprint for details on the setup 10.26434/chemrxiv-2022-r5wn0-v2 

import csv
import cantera as ct
import numpy as np

#######################################################################
# Input Parameters
#######################################################################

#enter the temperature in °C
tc=750
t = tc + 273.15  # convert to Kelvin

#parameters for the PFR model
length = 5e-2  # Length of the monolith m
N_Channel=195  # Number of channels of the monolith
radius=6.31e-4 # radius of the monolith m
GHSV = 30000/3600 # GHSV h^-1
V_flow=12.7/60/1000 #Volumetric flow rate
cross_area=np.pi/4*(radius)**2
area=2*np.pi*(radius)*length
pressure=1e5 # ambient pressure 
vol=length*cross_area
sccm=V_flow/N_Channel
total_cat_area = 0.527 #catalyst surface area measured with CO chemisorption
fcatgeo=total_cat_area/N_Channel/area
cat_area_per_channel=fcatgeo*area/vol

#input file containing the surface reaction mechanism
cti_file = 'chem.cti'
    
gas = ct.Solution(cti_file, 'gas')
gas.TPX = t, pressure, 'N2:73.7445, CH4(1):0.3, O2(6):1.206, CO2(2):12, H2O(3):12, H2(4):0.167, CO(5):0.5, C2H4(7):0.0325, C2H6(8):0.05'

# import the surface model
surf = ct.Interface(cti_file,'surface1', [gas])
surf.TP = t, pressure
surf.coverages = {'Pt(9)':1}

# The PFR will be simulated by a chain of 'NReactors' stirred reactors.
NReactors = 101
dt = 1.0
#Arrays to store the results
gas_frac=np.zeros((NReactors,gas.n_species))
surf_cov=np.zeros((NReactors,surf.n_species))
dist=np.zeros(NReactors)
        
rlen = length/(NReactors-1)
rvol = vol/(NReactors-1)

## catalyst area in one reactor
cat_area = cat_area_per_channel*rvol
mass_flow_rate =  sccm* gas.mean_molecular_weight*pressure/ct.gas_constant/273.15

TDY = gas.TDY
cov = surf.coverages
gas.TDY = TDY
r = ct.IdealGasReactor(gas, energy='off')
r.volume = rvol
upstream = ct.Reservoir(gas, name='upstream')
downstream = ct.Reservoir(gas, name='downstream')

rsurf = ct.ReactorSurface(surf, r, A=cat_area)
m = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)
v = ct.PressureController(r, downstream, master=m, K=1e-9)

sim = ct.ReactorNet([r])
sim.max_err_test_fails = 10000

## set relative and absolute tolerances on the simulation
sim.rtol = 1.0e-12
sim.atol = 1.0e-26

#The PFR is set up as a chain of CSTRs
for n in range(NReactors):
            # Set the state of the reservoir to match that of the previous reactor
            gas.TDY = r.thermo.TDY
            upstream.syncState()
            sim.reinitialize()
            sim.advance_to_steady_state(max_steps=1e6)
            dist[n] = n * rlen * 1.0e3   # distance in mm 
            gas_frac[n,:]=gas.X
            surf_cov[n,:]=surf.X 
            